<a href="https://colab.research.google.com/github/Liza-IITP/Dynamic_Pricing_Parking_Lots/blob/main/MODEL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh panel --quiet
import numpy as np
import pandas as pd
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

In [ ]:
df = pd.read_csv("data.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
vehicle_types = ["bike", "car", "cycle", "truck"]
for vt in vehicle_types:
    if f"VehicleType_{vt}" not in vehicle_dummies:
        vehicle_dummies[f"VehicleType_{vt}"] = 0
df = pd.concat([df, vehicle_dummies], axis=1)

vehicle_weights = {
    "VehicleType_bike": 0.2,
    "VehicleType_car": 1.0,
    "VehicleType_cycle": 0.1,
    "VehicleType_truck": 1.5
}
df["VehicleTypeWeight"] = sum(df[col] * wt for col, wt in vehicle_weights.items())

# Map traffic
traffic_map = {'low': 0, 'average': 1, 'high': 2}
df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map)

# Time features
df["DayOfWeek"] = df["Timestamp"].dt.dayofweek
df["TimeSlot"] = df["Timestamp"].dt.hour + df["Timestamp"].dt.minute / 60

def slot_weight(t):
    if 8 <= t < 10: return 0.8
    elif 10 <= t < 12: return 1.2
    elif 12 <= t < 14: return 1.5
    elif 14 <= t < 16: return 1.2
    else: return 1.0

df["SlotWeight"] = df["TimeSlot"].apply(slot_weight)

# Zone weight (mock rule: lat > 25.6 → downtown)
df["ZoneWeight"] = df["Latitude"].apply(lambda lat: 1.5 if lat > 25.6 else 1.0)

# Moving average and trend
df["OccRate_MA3"] = df.groupby("SystemCodeNumber")["OccupancyRate"].transform(lambda x: x.rolling(3, min_periods=1).mean())
df["OccRate_Trend"] = df.groupby("SystemCodeNumber")["OccupancyRate"].diff().fillna(0)

α, β, γ, δ, ε, τ = 1.0, 0.5, 0.3, 1.5, 1.0, 0.2
λ = 0.5
BasePrice = 10

df["RawDemand"] = (
    α * df["OccRate_MA3"] +
    β * df["QueueLength"] -
    γ * df["TrafficLevel"] +
    δ * df["IsSpecialDay"] +
    ε * df["VehicleTypeWeight"] +
    τ * df["OccRate_Trend"]
)
df["RawDemand"] *= df["SlotWeight"]
df["RawDemand"] *= df["ZoneWeight"]

# Normalize
min_d, max_d = df["RawDemand"].min(), df["RawDemand"].max()
df["NormalizedDemand"] = (df["RawDemand"] - min_d) / (max_d - min_d)
df["NormalizedDemand"] = df["NormalizedDemand"].clip(0, 1)
df["Model2Price"] = BasePrice * (1 + λ * df["NormalizedDemand"])
df["Model2Price_Smoothed"] = df.groupby("SystemCodeNumber")["Model2Price"].transform(lambda x: x.ewm(alpha=0.3).mean())

df.to_csv("parking_stream_model2.csv", index=False)

In [ ]:
class Model2Schema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRate: float
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_car: int
    VehicleType_cycle: int
    VehicleType_truck: int
    VehicleTypeWeight: float
    OccRate_MA3: float
    OccRate_Trend: float
    SlotWeight: float
    ZoneWeight: float
    RawDemand: float
    NormalizedDemand: float
    Model2Price: float
    Model2Price_Smoothed: float


In [ ]:

data = pw.demo.replay_csv("parking_stream_model2.csv", schema=Model2Schema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"))

In [ ]:
window = (
    data_time.windowby(
        pw.this.t,
        instance=(pw.this.SystemCodeNumber, pw.this.day),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        sum_price=pw.reducers.sum(pw.this.Model2Price_Smoothed),
        count_price=pw.reducers.count(pw.this.Model2Price_Smoothed),
    )
    .with_columns(
          AvgPrice_Model2 = (pw.this.sum_price / pw.this.count_price),
    )
)

In [ ]:
def plot(source):
    fig = bokeh.plotting.figure(
        height=450, width=900,
        title="Model 2 (Enhanced): Smoothed Demand-Based Price per Spot",
        x_axis_type="datetime"
    )
    fig.line("t", "AvgPrice_Model2", source=source, line_width=2, color="green")
    fig.scatter("t", "AvgPrice_Model2", source=source, size=6, color="red")
    fig.xaxis.axis_label = "Date"
    fig.yaxis.axis_label = "Avg Price"
    return fig

viz = window.plot(plot, sorting_col="t")
pn.panel(viz).servable()

In [ ]:

%%capture --no-display
pw.run()